## Basic backtesting

We've going to use [pandas-datareader](https://github.com/pydata/pandas-datareader/blob/master/pandas_datareader/tests/yahoo/test_yahoo.py) to collect data from Yahoo finance and pandas to work with this data.

In [1]:
from datetime import datetime
from pandas_datareader import data as web
import cufflinks as cf


start = datetime(2010, 1, 1)
end = datetime(2020, 6, 30)
ticker = "AAPL"
df = web.DataReader(ticker, "yahoo", start, end)

In [2]:
df.head(2)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,7.619643,7.520,7.611786,7.526072,352410800.0,6.503574
2010-01-04,7.660714,7.585,7.622500,7.643214,493729600.0,6.604801


In [3]:
df.tail(2)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-06-29,90.542503,87.82,88.312500,90.445000,130646000.0,90.282219
2020-06-30,91.495003,90.00,90.019997,91.199997,140223200.0,91.035858


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2642 entries, 2009-12-31 to 2020-06-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2642 non-null   float64
 1   Low        2642 non-null   float64
 2   Open       2642 non-null   float64
 3   Close      2642 non-null   float64
 4   Volume     2642 non-null   float64
 5   Adj Close  2642 non-null   float64
dtypes: float64(6)
memory usage: 144.5 KB


In [5]:
df["Adj Close"].normalize().iplot(
    kind="lines",
    title=ticker + " Adj Close (normalized)"
)

In [6]:
SMA1 = 40
SMA2 = 250

df["SMA1"] = df["Adj Close"].rolling(SMA1).mean()
df["SMA2"] = df["Adj Close"].rolling(SMA2).mean()

df.tail()

,High,Low,Open,Close,Volume,Adj Close,SMA1,SMA2
Date,,,,,,,,
2020-06-24,92.197502,89.629997,91.250000,90.014999,192623200.0,89.852997,80.719170,65.931383
2020-06-25,91.250000,89.392502,90.175003,91.209999,137522400.0,91.045845,81.205087,66.100333
2020-06-26,91.330002,88.254997,91.102501,88.407501,205256800.0,88.248390,81.583299,66.254512
2020-06-29,90.542503,87.820000,88.312500,90.445000,130646000.0,90.282219,82.041788,66.415662
2020-06-30,91.495003,90.000000,90.019997,91.199997,140223200.0,91.035858,82.493669,66.578170


In [7]:
columns = ["Adj Close", "SMA1", "SMA2"]
df[columns].iplot(title=ticker + " SMA")

## Creating positions

In [8]:
import numpy as np

df.dropna(inplace=True)
df["position"] = np.where(df['SMA1'] > df['SMA2'], 1, -1)

In [9]:
columns = ["Adj Close", "SMA1", "SMA2", "position"]

df[columns].iplot(
    secondary_y="position",
    title=ticker + " Positions",
)

## Getting returns from these positions

In [10]:
df["returns"] = np.log(df["Adj Close"] / df["Adj Close"].shift(1))

In [11]:
df.tail(2)

,High,Low,Open,Close,Volume,Adj Close,SMA1,SMA2,position,returns
Date,,,,,,,,,,
2020-06-29,90.542503,87.82,88.312500,90.445000,130646000.0,90.282219,82.041788,66.415662,1,0.022785
2020-06-30,91.495003,90.00,90.019997,91.199997,140223200.0,91.035858,82.493669,66.578170,1,0.008313


In [12]:
df.dropna(inplace=True)

In [13]:
df["returns"].iplot(kind="histogram")

Positions taken with EOD data get returns for the next trading day.

In [14]:
df["strategy"] = df["position"].shift(1) * df["returns"]
df.dropna(inplace=True)

np.exp(df[["returns", "strategy"]].sum())

returns     9.068095
strategy    5.950002
dtype: float64

In [15]:
df[["returns", "strategy"]].cumsum().apply(np.exp).iplot()

***